<a href="https://colab.research.google.com/github/kiranbeethoju/CHAI/blob/master/stock_predictions_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [171]:
import datetime
import pandas as pd
import numpy as np
import pylab as pl
import datetime
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
from matplotlib.collections import LineCollection
from pandas_datareader import data as wb
from sklearn import cluster, covariance, manifold

start = '1995-02-01'
end = '2020-06-25'

tickers = ['HINDPETRO.NS']

thelen = len(tickers)

price_data = []
for ticker in tickers:
    prices = wb.DataReader(ticker, start = start, end = end, data_source='yahoo')[['Open','Adj Close']]
    price_data.append(prices.assign(ticker=ticker)[['ticker', 'Open', 'Adj Close']])

#names = np.reshape(price_data, (len(price_data), 1))

names = pd.concat(price_data)
names.reset_index()

#pd.set_option('display.max_columns', 500)

open = np.array([q['Open'] for q in price_data]).astype(np.float)
close = np.array([q['Adj Close'] for q in price_data]).astype(np.float)
print(names)


                  ticker        Open   Adj Close
Date                                            
1996-01-02  HINDPETRO.NS   43.703701    1.294188
1996-01-03  HINDPETRO.NS   42.963001    1.285323
1996-01-05  HINDPETRO.NS   42.222198    1.294188
1996-01-08  HINDPETRO.NS   43.259300    1.245654
1996-01-09  HINDPETRO.NS   41.481499    1.262718
...                  ...         ...         ...
2020-06-22  HINDPETRO.NS  225.949997  228.137451
2020-06-23  HINDPETRO.NS  239.000000  227.181305
2020-06-24  HINDPETRO.NS  238.000000  218.384705
2020-06-25  HINDPETRO.NS  228.449997  212.121933
2020-06-26  HINDPETRO.NS  226.949997  221.109741

[6040 rows x 3 columns]


In [172]:
names = names.reset_index()
open = names.rename(columns={'Date':'ds','Open':'y'})
open = open[['ds','y']]
close = names.rename(columns={'Date':'ds','Adj Close':'y'})
close = close[["ds","y"]]

In [173]:
close

,ds,y
0,1996-01-02,1.294188
1,1996-01-03,1.285323
2,1996-01-05,1.294188
3,1996-01-08,1.245654
4,1996-01-09,1.262718
...,...,...
6035,2020-06-22,228.137451
6036,2020-06-23,227.181305
6037,2020-06-24,218.384705
6038,2020-06-25,212.121933


In [174]:
!pip install fbprophet

In [223]:
from fbprophet import Prophet
from plotly.graph_objs import *

df = open

def run(df):
  df = df[5900:]
  df.tail()
  #df = df[df['ds'].dt.dayofweek < 5]
  m = Prophet()
  m.fit(df)
  future = m.make_future_dataframe(periods=400)
  future.tail()
  forecast = m.predict(future)
  future = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
  future = future.rename(columns={"yhat":"yhat_future","yhat_lower":"yhat_lower_future","yhat_upper":"yhat_upper_future"})
  mixed = pd.merge(df,future,on="ds",how="inner")
  mixed.tail(30)
  avg = (mixed["yhat_future"].iloc[0]+mixed["yhat_lower_future"].iloc[0]+mixed["yhat_upper_future"].iloc[0])/3
  mixed["avg_val"] = ''
  for i in range(0, len(mixed)):
    avg = (mixed["yhat_future"].iloc[i]+mixed["yhat_lower_future"].iloc[i]+mixed["yhat_upper_future"].iloc[i])/3
    mixed['avg_val'].iloc[i] = avg
  mixed = mixed
  import plotly as py

  import plotly.graph_objects as go
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["y"],
                      mode='lines',
                      name='Actual Opening Values'))
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["yhat_future"],
                      mode='lines+markers',
                      name='Predicted Opening Values'))
  fig.show()
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["y"],
                      mode='lines',
                      name='Actual Opening Values'))
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["avg_val"],
                      mode='markers',
                      name='Predicted Opening Avg Values '))

  fig.show()
  return mixed

In [233]:
a = run(open)
b = run(close)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [238]:
a = a.rename(columns={"avg_val":"open_avg","y":"actual_open"})
b = b.rename(columns={"avg_val":"close_avg","y":"actual_close"})


In [239]:
c = pd.merge(a,b,on="ds",how="inner")

In [240]:
c

,ds,actual_open,yhat_future_x,yhat_lower_future_x,yhat_upper_future_x,open_avg,actual_close,yhat_future_y,yhat_lower_future_y,yhat_upper_future_y,close_avg
0,2019-12-03,282.950012,277.872289,262.897549,291.703586,277.491,262.367615,264.456141,250.669574,278.627016,264.584
1,2019-12-04,271.549988,276.626759,262.317217,290.895959,276.613,257.969330,263.308516,250.304033,277.126460,263.58
2,2019-12-05,268.500000,275.779195,261.175983,290.613531,275.856,254.240341,262.655663,246.702154,276.643124,262
3,2019-12-06,265.000000,272.986448,258.451568,286.862918,272.767,251.467514,261.504635,246.531262,275.641333,261.226
4,2019-12-09,263.000000,274.083753,259.425671,287.890861,273.8,257.108795,260.749210,246.156608,274.830019,260.579
...,...,...,...,...,...,...,...,...,...,...,...
135,2020-06-22,225.949997,213.354280,199.218261,228.026253,213.533,228.137451,205.314212,190.793142,219.127377,205.078
136,2020-06-23,239.000000,213.477025,199.628388,228.907890,214.004,227.181305,205.520445,191.749004,219.590211,205.62
137,2020-06-24,238.000000,213.287929,199.496778,227.314372,213.366,218.384705,205.481297,191.677879,219.347100,205.502
138,2020-06-25,228.449997,213.496800,197.801519,228.660039,213.319,212.121933,205.936921,191.980149,220.317124,206.078
